# PauliDecomposer - example 2
Decompose a slightly complicated entanglement witness into local measurements.
The witness matrix is no long a nice sparse matrix due to the rotation of its basis.
Optimized decomposer tries to find the eigenbasis to reduce the number of projections by
measuring in the eigenbasis of the witness operator.

In [1]:
import numpy as np
import KetSugar as ks
import PauliDecomposer as pd

In [2]:
#Bell state witness
base1_low = ks.BlochKet(np.pi/8, np.pi/8)
base1_high = ks.BlochKet(np.pi-np.pi/8, np.pi/8+np.pi)
base2_low = ks.BlochKet(np.pi/16, -np.pi/3)
base2_high = ks.BlochKet(np.pi-np.pi/16, -np.pi/3+np.pi)

bell_ket = (np.kron(base1_low, base2_high) + np.kron(base1_high, base2_low))*(0.5**.5)
bell_rho = ks.ketbra(bell_ket, bell_ket)

witness = np.eye(4)/2 - bell_rho
print(np.round(witness,3))
print(np.trace(bell_rho @ witness))

[[ 0.458+0.j     0.069+0.12j   0.128-0.053j  0.033+0.026j]
 [ 0.069-0.12j   0.042+0.j    -0.06 +0.454j -0.128+0.053j]
 [ 0.128+0.053j -0.06 -0.454j  0.042+0.j    -0.069-0.12j ]
 [ 0.033-0.026j -0.128-0.053j -0.069+0.12j   0.458+0.j   ]]
(-0.5000000000000002+1.474514954580286e-17j)


In [3]:
eye2 = np.eye(2) #identity matrix ... computational basis vector definition
decomposition = pd.decompose_operator_into_gen_pauli(witness, eye2, eye2)
#list coefficients
for i, (w_i, G_i) in enumerate(decomposition):
    print(i, pd.base10_to_base4(i,2), np.round(w_i,3))

0 [0, 0] (1+0j)
1 [0, 1] (-0+0j)
2 [0, 2] 0j
3 [0, 3] (-0+0j)
4 [1, 0] (-0-0j)
5 [1, 1] (0.831-0j)
6 [1, 2] (0.278+0j)
7 [1, 3] (-0.481+0j)
8 [2, 0] 0j
9 [2, 1] (0.513-0j)
10 [2, 2] (-0.053+0j)
11 [2, 3] (0.857+0j)
12 [3, 0] 0j
13 [3, 1] (0.213+0j)
14 [3, 2] (-0.959+0j)
15 [3, 3] (-0.186+0j)


In [4]:
#check validity
witness_check1 = np.zeros_like(witness)
norm = 2**(-2) #(1/2)^n ... where n is number of qubits
for w_i, G_i in decomposition:
    witness_check1 += norm*w_i*G_i

print(np.round(witness_check1,3))
print("Difference: ", np.sum(witness_check1 - witness))

[[ 0.458+0.j     0.069+0.12j   0.128-0.053j  0.033+0.026j]
 [ 0.069-0.12j   0.042+0.j    -0.06 +0.454j -0.128+0.053j]
 [ 0.128+0.053j -0.06 -0.454j  0.042+0.j    -0.069-0.12j ]
 [ 0.033-0.026j -0.128-0.053j -0.069+0.12j   0.458+0.j   ]]
Difference:  (-6.938893903907228e-17+0j)


In [5]:
#Find decomposition into projectors
coefficients = pd.decompose_operator_into_projectors(witness, eye2, eye2)
proj_operators = pd.spawn_generalized_pauli_projectors(eye2, eye2)
#check validity
witness_check_2 = pd.compose_operator_from_proj_weights(coefficients, proj_operators)
print("Difference:", np.sum(witness_check_2 - witness))
#list coefficients
for i, w_i in enumerate(coefficients):
    print(i, pd.base10_to_base6(i, 2), np.round(w_i.real, 3))

Difference: -6.938893903907228e-18j
0 [0, 0] 0.458
1 [0, 1] 0.042
2 [0, 2] 0.069
3 [0, 3] -0.069
4 [0, 4] -0.12
5 [0, 5] 0.12
6 [1, 0] 0.042
7 [1, 1] 0.458
8 [1, 2] -0.069
9 [1, 3] 0.069
10 [1, 4] 0.12
11 [1, 5] -0.12
12 [2, 0] 0.128
13 [2, 1] -0.128
14 [2, 2] -0.013
15 [2, 3] 0.013
16 [2, 4] 0.214
17 [2, 5] -0.214
18 [3, 0] -0.128
19 [3, 1] 0.128
20 [3, 2] 0.013
21 [3, 3] -0.013
22 [3, 4] -0.214
23 [3, 5] 0.214
24 [4, 0] 0.053
25 [4, 1] -0.053
26 [4, 2] -0.24
27 [4, 3] 0.24
28 [4, 4] -0.047
29 [4, 5] 0.047
30 [5, 0] -0.053
31 [5, 1] 0.053
32 [5, 2] 0.24
33 [5, 3] -0.24
34 [5, 4] 0.047
35 [5, 5] -0.047


In [7]:
#Find decomposition into projectors with smaller number of measurements
#by changing the measurement basis
opt_weights, opt_basis = pd.optimized_proj_decomposition(witness, 1e-4)
opt_proj_operators = pd.spawn_generalized_pauli_projectors(*opt_basis)
n_opt = sum(np.abs(opt_weights) > 1e-4)
n_default = sum(np.abs(coefficients) > 1e-4)
print("Optimized projection number:", n_opt)
print("Original projection number:", n_default)

#check validity
witness_check_3 = pd.compose_operator_from_proj_weights(opt_weights, opt_proj_operators)
print("Difference:", np.sum(witness_check_3 - witness))

#list the measurements, showing only projections above threshold
for i, w_i in enumerate(opt_weights):
    if abs(w_i) > 1e-4:
        print(i, pd.base10_to_base6(i, 2), np.round(w_i.real, 3))

Optimized projection number: 10
Original projection number: 36
Difference: (2.220446049250313e-16-1.2380527839228254e-18j)
1 [0, 1] 0.5
6 [1, 0] 0.5
14 [2, 2] 0.25
15 [2, 3] -0.25
20 [3, 2] -0.25
21 [3, 3] 0.25
28 [4, 4] -0.25
29 [4, 5] 0.25
34 [5, 4] 0.25
35 [5, 5] -0.25
